In [ ]:
from utils import *

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import pandas as pd

from ortools.sat.python import cp_model
from ortools.sat.python.cp_model import _SumArray

In [ ]:
values = getValues()
techs = getTechs()
tribes = getTribes()

In [ ]:
class GameState:
    turn = -1
    player = ""
    game = ""
    parent_state = None
    techs = {}
    previous_techs = {}
    vars = []
    tribe = ""
    cities = [] # 
    child_states = [] # GameStates
    raw_score = None
    full_score = None
    model = None
    solutions = None

    def __init__(self, turn, parent_state=None, previous_tech={}):
        self.turn = turn
        self.model = cp_model.CpModel()
        self.parent_state = parent_sate
        self.previous_tech = previous_tech

    def computeEndScore(self):
        self.

    def __repr__(self):
        return "GameState"
    
    def __str__(self):
        return "GameState turn %d, player %s" % (self.turn, self.player, self.game)

class Player:
    
    initialGameState = []

    def addScore(self):
        if len(initialGameState):
            pass
        pass

    def addTech(self):
        pass

    def addTribe(self, tribe):
        data = getTribe()[tribe]
        pass

In [ ]:
g = GameState()


In [ ]:
def solve(raw_score, full_score):
    model = cp_model.CpModel()
    eq_dict, sym_dict = addAllVars(model, values)

    buildScoreConstraint(model, eq_dict, full_score, True)
    buildScoreConstraint(model, eq_dict, raw_score, False)
    model.Add(_SumArray([sym_dict[e] for e in values['units'].keys()]) < 20)


    solver = cp_model.CpSolver()
    solution_printer = DataFrameSolutionAggregator(sym_dict.values())
    status = solver.SearchForAllSolutions(model, solution_printer)
    return solution_printer

In [ ]:
# superb_ruin
scores = [
    ([515, 515], []),
    ([565, 565], [440, 440]),
    ([590, 600], [450, 450]),
    ([855, 635], [700, 460]),
    ([1385, 670], [1230, 460]),
    ([1860, 1215], [1700, 990]),
    ([2230, 1815], [2010, 1530])
]

In [ ]:
player_scores = [
    [565, 440],
    [590, 450],
    [855, 700],
    [1385, 1230],
    [1860, 1700],
    [2230, 2010]
]

In [ ]:
player_2_scores = [
    [565, 440],
    [600, 450],
    [635, 460],
    [670, 460],
    [1215, 990],
    [1815, 1530]
]

In [ ]:
def divide_scores(scores):


    return selected_scores + divide_score(scores[1:])

In [ ]:
for i in range(len(player_scores) - 1):
    raw_delta = player_scores[i + 1][1] - player_scores[i][1]
    full_delta = player_scores[i + 1][0] - player_scores[i][0]
    print(full_delta, raw_delta)
    # sp = solve(raw_score=raw_delta, full_score=full_delta)
    # print(sp.get_data_frame().to_string())

In [ ]:
print(sp.get_data_frame())

# Equations: 

full score = score(units) + score(tech) + score(city) + score(vision)

raw score =  score(units) + score(tech) + score(city)

'remaining stars + spt + extra stars = stars(units) + stars(tech) + star(building)'

In [1]:
import re
from utils import * 
model = cp_model.CpModel()

past_turn = None
dictionnaries = {}
scores = {
    "start": [515, 390],
    "t0": [565, 440],
    "t1": [600, 450],
    "t2": [635, 460],
    "t3": [670, 460],
    "t4": [1215, 990],
    "t5": [1815, 1530]
}

for turn in ["start", "t0", "t1", "t2", "t3"]:#, "t4", "t5"]:
    dictionnaries[turn] = buildDictionnaries(model, turn)
    full_score = scores[turn][0] 
    raw_score = scores[turn][1]

    buildAllConstraints(model, turn, dictionnaries)

    setRawScore(model, turn, dictionnaries[turn]['scores'], raw_score)
    setFullScore(model, turn, dictionnaries[turn]['scores'], full_score)
    setTribe(model, turn, "Bardur", dictionnaries[turn]['tribes'])

    linkTurns(model, turn, dictionnaries)
    addInitialState(model, turn, dictionnaries)
    past_turn = turn


solver = cp_model.CpSolver()
output_variables = list([h for b, j in dictionnaries.items() for a,i in j.items() for h in i.values() if a != "equations"])
solution_printer = DataFrameSolutionAggregator(output_variables)
status = solver.SearchForAllSolutions(model, solution_printer)
print('Solve status: %s' % solver.StatusName(status))

Solve status: OPTIMAL


In [2]:
x = solution_printer.get_data_frame()

In [3]:
df = x.copy()

In [4]:
cleaned = chooseBestSolution(x.copy())

In [5]:
cleaned

,start_warrior,start_level-1,start_tier-1,start_claimed,start_revealed,start_hunting,start_Bardur,start_raw_score,start_full_score,t0_warrior,...,t3_Bardur,t3_raw_score,t3_full_score,t3_ruin,t3_ruin_giant,t3_ruin_stars,t3_level-2_spt,t3_level-2_count,t3_population_min,t3_population_max
0,1,1,1,9,25,1,1,390,515,1,...,1,460,670,0,0,0,1,1,2,4
1,1,1,1,9,25,1,1,390,515,1,...,1,460,670,0,0,0,1,1,2,4
2,1,1,1,9,25,1,1,390,515,1,...,1,460,670,0,0,0,1,1,2,4


In [6]:
def printScenario(full_df):
    regex = getScenarioRegex()
    selected_columns = [u for u in full_df.columns if sum(re.search(r, u) is not None for r in regex) != 0]
    #print(selected_columns)
    df = full_df[selected_columns]
    identical_columns = list(filter(lambda x: len(df[x].unique()) == 1, selected_columns))
    different_columns = list(filter(lambda x: len(df[x].unique()) != 1, selected_columns))
    turns = list(set(map(lambda u: u.split("_")[0], selected_columns)))
    for turn_i in sorted(turns):
        turn_string = ("Turn " + turn_i[1:] + ":" if turn_i.startswith("t") else "Start:")
        print(turn_string)
        print("  Common:")
        for element in identical_columns:
            if element.startswith(turn_i) and df.loc[0, element] != 0:
                print("   " + " ".join(element.split("_")[1:]) + ": " + str(df.loc[0, element]))
        if sum(e.startswith(turn_i) for e in different_columns) > 0:
            for i, row in df.iterrows():
                print(" Scenario " + str(i) + ":")
                print("  Specific:")
                for element in different_columns:
                    if element.startswith(turn_i):
                        print("   " + " ".join(element.split("_")[1:]) + ": " + str(row[element]))
printScenario(cleaned)

Start:
  Common:
   warrior: 1
   tier-1: 1
   claimed: 9
   revealed: 25
   hunting: 1
   Bardur: 1
Turn 0:
  Common:
   warrior: 1
   population: 2
   tier-1: 1
   claimed: 9
   revealed: 25
   claimed cst: 1
   hunting: 1
   Bardur: 1
   level-2 spt: 1
   level-2 count: 1
   population min: 2
   population max: 4
Turn 1:
  Common:
   tier-1: 1
   claimed: 9
   revealed: 30
   claimed cst: 1
   hunting: 1
   Bardur: 1
   level-2 spt: 1
   level-2 count: 1
   population min: 2
   population max: 4
 Scenario 0:
  Specific:
   warrior: 2
   population: 2
 Scenario 1:
  Specific:
   warrior: 2
   population: 2
 Scenario 2:
  Specific:
   warrior: 1
   population: 4
Turn 2:
  Common:
   tier-1: 1
   claimed: 9
   revealed: 35
   claimed cst: 1
   hunting: 1
   Bardur: 1
   level-2 spt: 1
   level-2 count: 1
   population min: 2
   population max: 4
 Scenario 0:
  Specific:
   warrior: 3
   population: 2
 Scenario 1:
  Specific:
   warrior: 2
   population: 4
 Scenario 2:
  Specific:
   wa

In [9]:
variants = printVariants(cleaned)

t1_warrior 2 [2 1]
t1_population 2 [2 4]
t2_warrior 2 [3 2]
t2_population 2 [2 4]
t3_warrior 2 [3 2]
t3_population 2 [2 4]


In [7]:
variants

[('t1_warrior', array([2, 1])),
 ('t1_population', array([2, 4])),
 ('t2_warrior', array([3, 2])),
 ('t2_population', array([2, 4])),
 ('t3_warrior', array([3, 2])),
 ('t3_population', array([2, 4]))]

In [21]:
# def prettyPrint(variants):
#     turns = list(set(map(lambda u: u[0].split("_")[0], variants)))
#     for turn_i in turns:
#         turn_string = ("Turn " + turn_i[1:] + ":" if turn_i.startswith("t") else "Start:")
#         print(turn_string)
#         for element, value in map(lambda u: (" ".join(u[0].split("_")[1:]), u[1]), filter(lambda u: u[0].startswith(turn_i), # variants)):
#             print(element + ":", ", ".join(map(lambda u: str(u), value)))
# prettyPrint(variants)

Turn 1:
warrior: 2, 1
population: 2, 4
Turn 2:
warrior: 3, 2
population: 2, 4
Turn 3:
warrior: 3, 2
population: 2, 4


In [6]:
printValueList(cleaned)

start_warrior 1 [1]
start_level-1 1 [1]
start_tier-1 1 [1]
start_claimed 1 [9]
start_revealed 1 [25]
start_hunting 1 [1]
start_Bardur 1 [1]
start_raw_score 1 [390]
start_full_score 1 [515]
t0_warrior 1 [1]
t0_level-1 1 [1]
t0_level-2 1 [1]
t0_population 1 [2]
t0_tier-1 1 [1]
t0_claimed 1 [9]
t0_revealed 1 [25]
t0_claimed_cst 1 [1]
t0_hunting 1 [1]
t0_Bardur 1 [1]
t0_raw_score 1 [440]
t0_full_score 1 [565]
t0_ruin 1 [0]
t0_ruin_stars 1 [0]
t0_level-2_spt 1 [1]
t0_level-2_count 1 [1]
t0_population_min 1 [2]
t0_population_max 1 [4]
t1_warrior 2 [2 1]
t1_giant 1 [0]
t1_level-1 1 [1]
t1_level-2 1 [1]
t1_population 2 [2 4]
t1_tier-1 1 [1]
t1_claimed 1 [9]
t1_revealed 1 [30]
t1_claimed_cst 1 [1]
t1_hunting 1 [1]
t1_Bardur 1 [1]
t1_raw_score 1 [450]
t1_full_score 1 [600]
t1_ruin 1 [0]
t1_ruin_giant 1 [0]
t1_ruin_stars 1 [0]
t1_level-2_spt 1 [1]
t1_level-2_count 1 [1]
t1_population_min 1 [2]
t1_population_max 1 [4]
t2_warrior 2 [3 2]
t2_giant 1 [0]
t2_level-1 1 [1]
t2_level-2 1 [1]
t2_populatio

64

In [ ]:
printValueList(x)